------
### 함수 호출 복습

In [ ]:
def foo(a, b, c):
    print(f'a:{a}')
    print(f'b:{b}')
    print(f'c:{c}')

In [ ]:
t = [3,4,5]
foo(t[0], t[1], t[2])

In [ ]:
foo(*t)

In [ ]:
t = {'a':1, 'b':2, 'c':3}
foo(**t)

------
### 클래스 복습 및 보강

In [ ]:
# 클래스 복습 및 보강

class Greeter(object):

    # 생성자
    def __init__(self, name):
        self.name = name  # 인스턴스 변수 선언

    # 인스턴스 메소드
    def greet(self, loud=False):
        if loud:
            print(f'HELLO, {self.name.upper()}!')
        else:
            print(f'Hello, {self.name}')
    
    # callable 객체로 만듦
    def __call__(self, loud=None):
        if loud is None: 
            print(f'my name is {self.name}')
        else:
            self.greet(loud)
    
    # len 프로토콜
    def __len__(self):
        #return 10
        #return len(self.name)
        return self.len_fn(self)
    
    # 인덱싱 프로토콜
    def __getitem__(self, position):
        return self.name[position]

g = Greeter('Fred')  # Greeter 클래스의 인스턴스 생성

In [ ]:
g.len_fn = lambda self: len(self.name)
len(g)

In [ ]:
g[1:3]

------
### 지난 숙제 검사

#### 셸 명령 실행

In [ ]:
!ls -l ..

In [ ]:
!ls -l ../homework/*

In [ ]:
es = !ls -l ../homework/*

In [ ]:
es

------
#### 리스트 comprehension 및 문자열 조작

In [ ]:
es = [e for e in es if e.endswith('.ipynb')] 
es

In [ ]:
es = [e.replace('naheenosaur', '김나희') for e in es]
es

In [ ]:
es = [e.split(' ')[-1] for e in es]
es

In [ ]:
es = [e.split('.')[0] for e in es]
es

In [ ]:
es = [e.split('_')[-1] for e in es]
es

In [ ]:
', '.join(es)

In [ ]:
es[:3] + ['...'] + es[-3:]

In [ ]:
try:
    import pandas as pd
except: 
    !pip install pandas
    import pandas as pd
    
df = pd.read_csv('명단.txt')
df

In [ ]:
miss = set(df['이름'].values) - set(es)
print(f'미 제출자: {", ".join(miss)}')

------
### 폴더 순회 

In [ ]:
from pathlib import Path
BASE = Path('../homework/20200129/')
files = list(BASE.iterdir())
files

In [ ]:
files = [f for f in files if f.suffix =='.ipynb']
files

In [ ]:
import json
def solution_code(path):
    with open(path) as f:
        try:
            txt = json.load(f)
        except:
            with open(path) as f:
                es = f.readlines()
            txt = {'cells':[{'cell_type':'code', 'source':es}]}
            
    cells = [c for c in txt['cells'] if c['cell_type'] == 'code']
    sources = [c['source'] for c in cells]

    def get_code(source):
        source = [line for line in source if not line.startswith('!')]
        source = [line for line in source if not 'print' in line]
        source = [line for line in source if not 'gid.googleimagesdownload().download' in line]
        
        s = [idx for idx, line in enumerate(source) if line.startswith('def download')]
        if len(s) == 0:
            return None
        
        r = [idx for idx, line in enumerate(source) if 'return' in line]
        if len(r) == 0:
            return None
        
        s,  r = s[0],  r[-1]
        
        source[r] = "    return time_range, out_folder \n"
        source = source[:r+1]
        return ''.join(source)

    funcs = [get_code(s) for s in sources] 
    funcs = [f for f in funcs if f is not None]
    assert len(funcs) == 1
    func  = funcs[0]
    return func

def get_answer(path):
    code = solution_code(path)
    #py = path.with_suffix('.py').name
    py = 'homework.py'
    with open(py, 'w') as f:
        f.write(code)
    
    import importlib
    hw = importlib.import_module('homework')
    hw = importlib.reload(hw)
    args = [hw.download(m) for m in range(1, 13)]
    person = path.stem.split('_')[-1]
    return [person] + list(zip(*args)) 

ass = [get_answer(p) for p in files]
ass_1 = sorted([(o[1].replace(' ', ''), p) for p, o, t in ass])
ass_2 = sorted([(t[1], p) for p, o, t in ass])


In [ ]:
ass_1

In [ ]:
set_1 = {s for s,p in ass_1}
len(ass_1), len(set_1)

In [ ]:
set_1 = {''.join(e).replace(' ', '').replace('}', '} ')[:100] for e in set_1}
print(len(set_1))
set_1

------
## 숙제

* 지난 번 숙제를 통해 다운 받은 이미지에 대한 메타 데이터를 추출하세요.  

* 추출된 메타 데이터에 대한 통계값을 계산하세요.  

* 추출된 메터 데이터는 별도 파일로 저장하고(json 또는 csv 포멧)

   계산된 통계값은 화면에 출력하세요 (최대한 이쁘게)
   
   
* 파일별로 추출할 메타 데이터는 다음과 같습니다.
  * 파일 이름
  * 상위 폴더명
  * 이미지 확장자 
  * 이미지 해상도  
    
* 계산할 통계값은 다음과 같습니다.
  * 이미지 해상도 통계
    * 이미지 해상도 종류
    * 각 이미지 해상도별 사진 갯수
    * 이미지 넓이(픽셀 높이 x 픽셀 넓이)의 대표값
        * 평균값, 최소값, 최대값, 최빈값, 중앙값
  * 이미지 확장자 통계
    * 확장자 종류
    * 각 이미지 확장자별 사진 갯수
  * 폴더별 통계
    * 각 폴더별 이미지 갯수
 
  
* 이미지 해상도 알아내는 방법은 아래 패키지를 이용하세요. 
  * https://github.com/shibukawa/imagesize_py


In [49]:
from pathlib  import Path
from collections import Counter
import json
import statistics
try: 
    import imagesize
except:
    !pip install imagesize
    import imagesize
    
# 폴더 내의 파일들에 대해 메타 정보 추출하기
def extract_metas(folder):
    def extract_meta(path):
        path = Path(path)
        return {
            'name': path.name,
            'parent': path.parent,
            'suffix': path.suffix,
            'size': imagesize.get(path)
        }
    return [extract_meta(f) for f in Path(folder).iterdir() if not f.is_dir()]

# 메타 정보 폴더별로 추출하기
metas = [extract_metas(d) for d in Path('downloads').iterdir() if d.is_dir()]

# 폴더별로 추출된 것 합치기: flatten
metas = sum(metas, [])

# 메타 정보 통계값을 출력하세요.
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i+n]
        
def print_statistics(metas):
    print('1. 이미지 해상도 통계')
    print('1-1. 이미지 해상도 종류는 뭐가 있을까?')
    pixels = [meta['size'] for meta in metas]
    pixels = Counter(pixels)
    keysList = list(pixels.keys())

    for i in range(0, len(keysList), 5):
        l = keysList[i:i+5]
        #print(l, sep='\n')
    
    print('\n\n')
    print('1-2. 각 이미지 해상도별 개수를 알아보자')
    kvList = list(element for element in pixels.items())
    for i in range(0, len(kvList), 5):
        l_ = kvList[i:i+5]
        #print(l_, sep='\n')
 
    print('\n\n')
    print('1-3. 각 이미지에 대한 넓이의 대표값구하기')
    sizes = [w*h for w,h in pixels]
    print('1-3-1 평균값: ', statistics.mean(sizes))
    print('1-3-2 최소값: ', min(sizes))
    print('1-3-3 최대값: ', max(sizes))
    print('1-3-4 최빗값: ', (lambda l: l.most_common(1))(pixels))
    print('1-3-5 중앙값: ', statistics.median(sizes))
    
    print('\n\n')
    print('2. 이미지 확장자 통계')
    print('2-1. 이미지 확장자 종류를 알아보자.')
    suffixes = [meta['suffix'] for meta in metas]
    suffixes = Counter(suffixes)
    keySuffix = list(suffixes.keys())
    
    for i in range(0, len(keySuffix), 5):
        l = keySuffix[i:i+5]
        print(l, sep='\n')
    
    print('\n\n')
    print('2-2. 각 이미지 확장자 별 개수를 알아봅시다')
    kvList = list(element for element in suffixes.items())
    print(kvList)
    
    print('\n\n')
    parents = []
    print('3. 폴더별 이미지 개수는?')
    for meta in metas:
        m = str(meta['parent']).split('\\')
        parent = m[-1]
        parents.append(parent)
    
    parentCnt = Counter(parents)
    for k,v in parentCnt.items():
        print(k, v)
    
# 메타 정보를 저장하는 함수를 작성하세요
def save_meta(metas, write_path):
    '''
    여기를 작성하세요.
    write_path 경로에 메타 데이터 리스트를 저장하세요
    json 또는 csv 포멧으로 저장하세요
    '''
    
print_statistics(metas) 

1. 이미지 해상도 통계
1-1. 이미지 해상도 종류는 뭐가 있을까?



1-2. 각 이미지 해상도별 개수를 알아보자



1-3. 각 이미지에 대한 넓이의 대표값구하기
1-3-1 평균값:  761364.3740259741
1-3-2 최소값:  1
1-3-3 최대값:  12192768
1-3-4 최빗값:  [((640, 640), 21)]
1-3-5 중앙값:  360000



2. 이미지 확장자 통계
2-1. 이미지 확장자 종류를 알아보자.
['.jpg', '.png', '.gif', '.JPEG', '.JPG']
['.PNG', '.jpeg', '.svg']



2-2. 각 이미지 확장자 별 개수를 알아봅시다
[('.jpg', 356), ('.png', 223), ('.gif', 16), ('.JPEG', 1), ('.JPG', 14), ('.PNG', 26), ('.jpeg', 9), ('.svg', 2)]



3. 폴더별 이미지 개수는?
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downloads', '한글폰트_2018_01']
['downlo